In [5]:
# !pip install keras-tuner
# !pip install tensorflow
# !pip install tensorflow-macos
# !pip install tensorflow-metal
# !pip install scikit-learn

# 1- Importing Libraries

In [1]:
import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os

# 2- Loading Dataset

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
X_train = x_train.reshape(-1,28, 28, 1) 
X_test = x_test.reshape(-1,28,28,1)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(48000, 28, 28, 1)
(48000, 10)
(12000, 28, 28, 1)
(12000, 10)


# 3- DeepTuner

In [5]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32]), **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}
            

# class DeepTuner(kt.Tuner):
#     def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
#         model = self.hypermodel.build(trial.hyperparameters)
#         model.fit(X, y, batch_size=trial.hyperparameters.Choice('batch_size', [16, 32]), **fit_kwargs)

#         X_val, y_val = validation_data
#         eval_scores = model.evaluate(X_val, y_val)
        
#         # 메트릭 이름을 기반으로 'accuracy' 메트릭의 인덱스 찾기
#         accuracy_index = model.metrics_names.index('accuracy') if 'accuracy' in model.metrics_names else None
        
#         # 'accuracy' 메트릭이 존재하지 않는 경우 예외 처리
#         if accuracy_index is None:
#             raise ValueError("The model's metrics do not include 'accuracy'. Please ensure 'accuracy' is included in the model's compile metrics.")
        
#         # 'accuracy' 메트릭 값 반환
#         return {'accuracy': eval_scores[accuracy_index]}

# 4- Building Model

In [6]:
from ipaddress import summarize_address_range
from pyexpat import model


def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape = X_train.shape[1:], name = 'inputs'))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
              model.add(tf.keras.layers.Conv2D(hp.Int(
                  'units_{i}'.format(i=i), min_value=32, max_value=128, step=5), (3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    for i in range(hp.Int('n_connections', 1, 3)):
        model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes',
                                  values=[32,64,128, 256]), activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))
    model.compile(optimizer = 'adam',loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

<module 'pyexpat.model'>


# 5- Keras Tuner

In [7]:
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='my_keras_tuner')

my_keras_tuner.search(
    X_train, y_train, validation_data=(X_val, y_val), epochs=10)

2024-04-09 15:30:27.540712: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-04-09 15:30:27.540738: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-04-09 15:30:27.540744: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-04-09 15:30:27.540789: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-09 15:30:27.540804: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
7                 |7                 |num_layers
32                |32                |units_0
2                 |2                 |n_connections
64                |64                |n_nodes

Epoch 1/3


2024-04-09 15:30:28.267212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


3000/3000 ━━━━━━━━━━━━━━━━━━━━ 75s 24ms/step - accuracy: 0.9010 - loss: 0.3411
Epoch 2/3
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 73s 24ms/step - accuracy: 0.9784 - loss: 0.0735
Epoch 3/3
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 74s 25ms/step - accuracy: 0.9851 - loss: 0.0496
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9759 - loss: 0.0880


FatalValueError: Expected the returned dictionary from Tuner.run_trial() to have the specified objective, accuracy, as one of the keys. Received: {'loss': 0.07961367070674896, 'compile_metrics': 0.9782500267028809}.

# 6- Checking Model

In [8]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 22, 22, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 20, 20, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 18, 18, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 14, 14, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │       401,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputs (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 462,090 (1.76 MB)

 Trainable params: 462,090 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

# 7- Training Optimum Model

In [9]:
model.fit(X_train, y_train, batch_size=32, epochs = 5)

Epoch 1/5


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 40s 25ms/step - accuracy: 0.8908 - loss: 0.3410
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 37s 25ms/step - accuracy: 0.9807 - loss: 0.0642
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 37s 25ms/step - accuracy: 0.9853 - loss: 0.0496
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 38s 25ms/step - accuracy: 0.9878 - loss: 0.0405
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 38s 25ms/step - accuracy: 0.9900 - loss: 0.0315


# 8- Evaluating Model

In [10]:
model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9808 - loss: 0.0665


[0.04832499101758003, 0.98580002784729]

# 9- Saving Model

In [11]:
save_path = '/Users/kenny_jung/aiffel/data/mlops/best_model/'
fname = os.path.join(save_path, 'model.keras')  # Changed to include .keras extension
model.save(fname)

In [12]:
save_path = '/Users/kenny_jung/aiffel/data/mlops/best_model'
fname = os.path.join(save_path, 'model.h5')  # Changed to include .h5 extension
model.save(fname)

# 10- Light Model

In [13]:
# 모델 파일 경로
model_path = '/Users/kenny_jung/aiffel/data/mlops/best_model/model.keras'

# 모델 로드
model = tf.keras.models.load_model(model_path)

# 로드된 모델 요약 출력
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 22, 22, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 20, 20, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 18, 18, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 14, 14, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │       401,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputs (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,386,272 (5.29 MB)

 Trainable params: 462,090 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 924,182 (3.53 MB)

# 11- Transformation to tflite

In [15]:
print(tf.__version__)

2.16.1


In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/b9/d_8nll_s47g6w2x0spndwwl80000gn/T/tmpnzylhwfv/assets


INFO:tensorflow:Assets written to: /var/folders/b9/d_8nll_s47g6w2x0spndwwl80000gn/T/tmpnzylhwfv/assets


: 